In [174]:
import pandas as pd
import os
import numpy as np
# read data and extract variables simultaneously
xpt_files = ["ALQ_I.XPT", "DEMO_I.XPT", "OCQ_I.XPT", "DIQ_I.XPT", 
             "DR1IFF_I.XPT", "SLQ_I.XPT", "DR1TOT_I.XPT"]
base = ""
vars = [["SEQN", "ALQ130"],
        ["SEQN", "RIAGENDR", "RIDAGEYR"],
        ["SEQN", "OCQ260"],
        ["SEQN", "DIQ010"],
        ["SEQN", "DR1ISUGR", "DR1ITFAT"],
        ["SEQN", "SLD012"],
        ["SEQN", "DR1TSUGR", "DR1TTFAT"]
    ]

# STORE files in a list
da = []
for idf, fn in enumerate(xpt_files):
    df = pd.read_sas(os.path.join(base, fn))
    df = df.loc[:, vars[idf]]
    da.append(df)

In [175]:
# label the datasets
alcohol = da[0]
demo = da[1]
diabete = da[3]
dietary = da[4]
occupation = da[2]
sleep = da[5]
total_nutrient = da[6]

In [176]:
# data cleaning - missing, re-categorize,

# diebete
for i in range(diabete.shape[0]):
    if diabete.iloc[i][1] == 2.0:
        diabete.iloc[i][1] = int(0)
    elif diabete.iloc[i][1] == 1.0:
        diabete.iloc[i][1] = int(diabete.iloc[i][1])
    else:
        diabete.iloc[i][1] = np.nan
        
# alcohol
for j in range(alcohol.shape[0]):
    if alcohol.iloc[j][1] in [777.0, 999.0]:
        alcohol.iloc[j][1] == np.nan
    
# occupation
for k in range(occupation.shape[0]):
    if occupation.iloc[k][1] in [77.0, 99.0, 6.0]:
        occupation.iloc[k][1] = np.nan
    elif occupation.iloc[k][1] in [2.0, 3.0, 4.0]:
        occupation.iloc[k][1] = 2.0
    elif occupation.iloc[k][1] == 5.0:
        occupation.iloc[k][1] = 3.0

# dietary
dietary = dietary.groupby('SEQN').sum()
dietary.columns = ["Tot_sugar", "Tot_fat"]
dietary = dietary.reset_index()

In [177]:
# merge tables
table = pd.merge(left=alcohol,right=demo, left_on='SEQN', right_on='SEQN')
table = pd.merge(left=table,right=diabete, left_on='SEQN', right_on='SEQN')
table = pd.merge(left=table,right=dietary, left_on='SEQN', right_on='SEQN')
table = pd.merge(left=table,right=occupation, left_on='SEQN', right_on='SEQN')
table = pd.merge(left=table,right=sleep, left_on='SEQN', right_on='SEQN')
table = table.dropna()
# 2087 rows left after removing nan

In [178]:
# contingency table - gender
pd.crosstab(table["DIQ010"], table["RIAGENDR"])

RIAGENDR,1.0,2.0
DIQ010,,
0.0,1011,824
1.0,108,74


In [179]:
# contingency table - occupation
pd.crosstab(table["DIQ010"], table["OCQ260"])

OCQ260,1.0,2.0,3.0
DIQ010,,,
0.0,1380,268,187
1.0,122,30,30


In [180]:
# Change numeric to dummy
# handle multi-level categorical variable - OCQ260
occ = pd.get_dummies(table['OCQ260'], prefix='OCQ260')
gender = pd.get_dummies(table['RIAGENDR'], prefix='RIAGENDR')
occ.columns = ['business', 'government', 'self-employee']
gender.columns = ['male','female']
cols_to_keep = ['ALQ130', "RIDAGEYR","SLD012", "Tot_sugar", "Tot_fat"]

df_temp = table[cols_to_keep].join(occ.loc[: , 'government' : ])
df_temp = df_temp.join(gender['female'])

In [181]:
X = df_temp.iloc[:,:]
# add intercept manually
X['intercept'] = 1.0
y = table['DIQ010']

In [182]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.271103
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 DIQ010   No. Observations:                 2017
Model:                          Logit   Df Residuals:                     2008
Method:                           MLE   Df Model:                            8
Date:                Fri, 22 Nov 2019   Pseudo R-squ.:                  0.1055
Time:                        22:20:57   Log-Likelihood:                -546.82
converged:                       True   LL-Null:                       -611.31
Covariance Type:            nonrobust   LLR p-value:                 4.602e-24
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
ALQ130           -0.0444      0.044     -0.999      0.318      -0.132       0.043
RIDAGEYR          0.